# Interactive Multiverse Oscillator Model

This notebook lets you explore the toy model where universes are coupled like oscillators,
with black-hole-like junctions acting as diode thresholds.

Adjust the parameters below to see how coherence reinforcement emerges.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from scipy.integrate import solve_ivp

def multiverse_model(t, y, omega, lam, C, threshold):
    # y = [Re1, Im1, Re2, Im2, Re3, Im3]
    dydt = []
    for i in range(3):
        Re, Im = y[2*i], y[2*i+1]
        z = Re + 1j*Im
        # Stuart-Landau dynamics with coupling
        dzdt = (lam[i] + 1j*omega[i])*z - (abs(z)**2)*z
        # Coupling from neighbors
        for j in range(3):
            if j != i:
                coupling = C * (y[2*j] + 1j*y[2*j+1] - z)
                # diode-like gating
                if abs(coupling) > threshold:
                    dzdt += coupling
        dydt.extend([dzdt.real, dzdt.imag])
    return dydt

def run_simulation(C=0.1, threshold=0.2, w1=1.0, w2=1.05, w3=0.95, l1=0.1, l2=0.1, l3=0.1):
    omega = [w1, w2, w3]
    lam = [l1, l2, l3]
    y0 = [1, 0, 0.5, 0.1, 0.2, -0.2]  # initial conditions
    t_span = (0, 100)
    t_eval = np.linspace(*t_span, 1000)
    sol = solve_ivp(multiverse_model, t_span, y0, t_eval=t_eval, args=(omega, lam, C, threshold))

    plt.figure(figsize=(10,5))
    for i in range(3):
        Re, Im = sol.y[2*i], sol.y[2*i+1]
        amp = np.sqrt(Re**2 + Im**2)
        plt.plot(sol.t, amp, label=f'Oscillator {i+1}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude (|z|)')
    plt.legend()
    plt.title('Coherence Reinforcement Across Universes')
    plt.show()

interact(run_simulation,
         C=FloatSlider(min=0, max=1, step=0.05, value=0.1),
         threshold=FloatSlider(min=0, max=1, step=0.05, value=0.2),
         w1=FloatSlider(min=0.5, max=1.5, step=0.05, value=1.0),
         w2=FloatSlider(min=0.5, max=1.5, step=0.05, value=1.05),
         w3=FloatSlider(min=0.5, max=1.5, step=0.05, value=0.95),
         l1=FloatSlider(min=0, max=0.5, step=0.05, value=0.1),
         l2=FloatSlider(min=0, max=0.5, step=0.05, value=0.1),
         l3=FloatSlider(min=0, max=0.5, step=0.05, value=0.1))